## Bic Vu's Capstone Project
#### ML Guild - Apprentice Cohort 11
#### Notebook 2
2023-06-10

In [19]:
import pandas as pd
import pandas_profiling as pf
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import random
%matplotlib inline

pd.set_option('display.max_columns', None)
pd.set_option("display.precision", 12) # Expand number char shown to 12 instead of scientific notation

from datetime import date
today = date.today()

In [2]:
# Create calculate_feature_statistics

from pandas.api.types import is_numeric_dtype

def calculate_feature_statistics(data_frame):
    results = pd.DataFrame()
    for column in data_frame.columns:
        column_data = data_frame[column]
        if not is_numeric_dtype(column_data): continue
        num_zeros = (column_data == 0).sum()
        prop_zeros = num_zeros / len(column_data)
        num_positives = (column_data > 0).sum()
        prop_positives = num_positives / len(column_data)
        num_negatives = (column_data < 0).sum()
        prop_negatives = num_negatives / len(column_data)
        num_missing = column_data.isnull().sum()
        prop_missing = num_missing / len(column_data)
        #most_frequent_value = column_data.mode()[0]
        nunique_value = column_data.nunique()
        prop_nunique = nunique_value / len(column_data)
        min_value = column_data.min()
        max_value = column_data.max()
        median_value = column_data.median()
        feature_stats = pd.Series([num_zeros, prop_zeros, num_positives, prop_positives,
                                   num_negatives, prop_negatives, num_missing, prop_missing,
                                   #most_frequent_value, 
                                   nunique_value, prop_nunique, min_value, max_value, median_value],
                                  index=['num_zeros', 'prop_zeros', 'num_positives', 'prop_positives',
                                         'num_negatives', 'prop_negatives', 'num_missing', 'prop_missing',
                                         #'most_frequent_value', 
                                         'nunique_value', 'prop_nunique', 'min_value', 'max_value', 'median_value'])
        results[column] = feature_stats
    return results.T

In [3]:
# Create shortcut functions to display max rows and columns
def maxcol():
    pd.set_option('display.max_columns', None)
    return

def maxcol_reset():
    pd.reset_option('display.max_columns', None)
    return

def maxrow():
    pd.set_option('display.max_rows', None)
    return

def maxrow_reset():
    pd.reset_option('display.max_rows', None)
    return

### Import raw Social Capital data

In [4]:
filepath = "data/social_capital_zip.csv"
sc_samp = pd.read_csv(filepath, nrows=100)
sc_samp.columns

Index(['zip', 'county', 'num_below_p50', 'pop2018', 'ec_zip', 'ec_se_zip',
       'nbhd_ec_zip', 'ec_grp_mem_zip', 'ec_high_zip', 'ec_high_se_zip',
       'nbhd_ec_high_zip', 'ec_grp_mem_high_zip', 'exposure_grp_mem_zip',
       'exposure_grp_mem_high_zip', 'nbhd_exposure_zip', 'bias_grp_mem_zip',
       'bias_grp_mem_high_zip', 'nbhd_bias_zip', 'nbhd_bias_high_zip',
       'clustering_zip', 'support_ratio_zip', 'volunteering_rate_zip',
       'civic_organizations_zip'],
      dtype='object')

In [5]:
filepath = "data/social_capital_zip.csv"

cols = ["zip", "num_below_p50", "ec_zip", "nbhd_ec_zip", "exposure_grp_mem_zip", "nbhd_exposure_zip", "clustering_zip", 'volunteering_rate_zip', 'civic_organizations_zip']
types = {"zip": str}
sc = pd.read_csv(filepath, usecols=cols, dtype=types)

print(sc.shape)
sc.tail(100)

(23028, 9)


,zip,num_below_p50,ec_zip,nbhd_ec_zip,exposure_grp_mem_zip,nbhd_exposure_zip,clustering_zip,volunteering_rate_zip,civic_organizations_zip
22928,99216,2584.053627014160,0.85681999,0.73123002,0.97567999,0.81203002,0.086939998,0.064539999,0.0044000000
22929,99217,2090.080810546875,0.78706002,0.58104002,0.91451001,0.73140001,0.087669998,0.066299997,0.0066300002
22930,99218,963.998716354370,0.87724000,0.78653997,1.01434000,0.90186000,0.088289998,0.064070001,0.0141600000
22931,99223,1982.388222217560,0.96315002,0.93023998,1.08135000,1.07786000,0.085359998,0.068410002,0.0099999998
22932,99224,1187.107990026474,0.86550999,0.69260001,1.01274000,0.93377000,0.088110000,0.069559999,0.0076299999
...,...,...,...,...,...,...,...,...,...
23023,99901,1192.299808979034,0.99517000,0.88014001,1.01237000,0.94761997,0.134730000,0.118830000,0.0299900010
23024,99921,365.768661499023,0.87976998,0.74554998,0.88303000,0.81680000,0.155610000,0.084040001,0.0321500000
23025,99925,154.513839721680,NaN,NaN,NaN,NaN,0.146579340,0.123960490,0.0277283400
23026,99926,311.014251708984,0.87888002,0.81081003,0.84147000,0.80694002,0.252740000,0.142910000,0.0112500000


In [6]:
sc = sc.dropna().reset_index(drop=True)
print(sc.shape)
sc.head()

(14271, 9)


,zip,num_below_p50,ec_zip,nbhd_ec_zip,exposure_grp_mem_zip,nbhd_exposure_zip,clustering_zip,volunteering_rate_zip,civic_organizations_zip
0,1001,995.787467956543,0.88156998,1.51095000,1.12959,1.50590000,0.105720000,0.056499999,0.0108000000
1,1002,1312.117076873779,1.18348000,0.97759998,1.36818,1.20281990,0.103400000,0.149510000,0.0368800010
2,1005,381.519744873047,1.15543000,1.46491000,1.31876,1.46396990,0.105540000,0.158620000,0.0216300000
3,1007,915.396667480469,1.19240000,1.17985000,1.30741,1.23108990,0.103910000,0.130530000,0.0168999990
4,1013,2616.550354003906,0.69744003,0.53930002,0.87842,0.67602003,0.086479999,0.061910000,0.0096899997


In [7]:
sc.dtypes

zip                         object
num_below_p50              float64
ec_zip                     float64
nbhd_ec_zip                float64
exposure_grp_mem_zip       float64
nbhd_exposure_zip          float64
clustering_zip             float64
volunteering_rate_zip      float64
civic_organizations_zip    float64
dtype: object

In [8]:
# Fill in zip codes with less than 5 digits with leading 0
sc["zip"] = sc["zip"].apply(lambda x: x.zfill(5))
sc.head()

,zip,num_below_p50,ec_zip,nbhd_ec_zip,exposure_grp_mem_zip,nbhd_exposure_zip,clustering_zip,volunteering_rate_zip,civic_organizations_zip
0,01001,995.787467956543,0.88156998,1.51095000,1.12959,1.50590000,0.105720000,0.056499999,0.0108000000
1,01002,1312.117076873779,1.18348000,0.97759998,1.36818,1.20281990,0.103400000,0.149510000,0.0368800010
2,01005,381.519744873047,1.15543000,1.46491000,1.31876,1.46396990,0.105540000,0.158620000,0.0216300000
3,01007,915.396667480469,1.19240000,1.17985000,1.30741,1.23108990,0.103910000,0.130530000,0.0168999990
4,01013,2616.550354003906,0.69744003,0.53930002,0.87842,0.67602003,0.086479999,0.061910000,0.0096899997


### Opt 1: Zip to Census Tract
Since there are fewer census tracts to zip codes; converting Social Capital zipcode data to census tracts will aggregate some zip codes.

In [9]:
cols = ["zip", "tract"]
zip_ct = pd.read_excel("data/ZIP_TRACT_122021.xlsx", usecols=cols)
zip_ct.head()

,zip,tract
0,683,72023830102
1,683,72125840700
2,683,72125840400
3,683,72125840600
4,683,72121960300


In [10]:
# There are fewer census tracts than zip codes
zip_dup = len(zip_ct["zip"])-len(zip_ct["zip"].drop_duplicates())
print(zip_dup)
tract_dup = len(zip_ct["tract"])-len(zip_ct["tract"].drop_duplicates())
print(tract_dup)

132853
98705


In [11]:
zip_ct = zip_ct.astype(str)
zip_ct.dtypes

zip      object
tract    object
dtype: object

In [12]:
# Format census tracts and zip codes with leading zeroes
zip_ct["tract"] = zip_ct["tract"].apply(lambda x: x.zfill(11))
zip_ct["zip"] = zip_ct["zip"].apply(lambda x: x.zfill(5))
zip_ct.tail()

,zip,tract
172172,99163,53075000500
172173,99163,53075000200
172174,99163,53075001000
172175,78635,48171950100
172176,78635,48031950100


In [13]:
zip_ct = zip_ct.rename(columns={'tract':'census_tract'})

### Convert Social Capital zipcode to census tracts

In [14]:
# Join social capital data with geographic converter on zip
# This will create a lot of duplicates

print(sc.shape)
sc = sc.merge(zip_ct, on="zip", how="left")
print(sc.shape)
sc.head()

(14271, 9)
(117978, 10)


,zip,num_below_p50,ec_zip,nbhd_ec_zip,exposure_grp_mem_zip,nbhd_exposure_zip,clustering_zip,volunteering_rate_zip,civic_organizations_zip,census_tract
0,01001,995.787467956543,0.88156998,1.51095,1.12959,1.5059,0.10572,0.056499999,0.0108,25013813209
1,01001,995.787467956543,0.88156998,1.51095,1.12959,1.5059,0.10572,0.056499999,0.0108,25013813208
2,01001,995.787467956543,0.88156998,1.51095,1.12959,1.5059,0.10572,0.056499999,0.0108,25013813204
3,01001,995.787467956543,0.88156998,1.51095,1.12959,1.5059,0.10572,0.056499999,0.0108,25013813207
4,01001,995.787467956543,0.88156998,1.51095,1.12959,1.5059,0.10572,0.056499999,0.0108,25013813206


### Import HMDA data and merge with social capital data

In [15]:
# Import HMDA data
hmda = pd.read_parquet("data/output/hmda_2021_ct_group_2023-06-10")
print(hmda.shape)
hmda.head()

(72681, 9)


,census_tract,approval,loan_amount,property_value,income,tract_minority_population_percent,income_level,dti_level,derived_race
0,01001020100,0.764705882353,190294.117647058825,247941.176470588223,86.078431372549,12.58,upper,36,White
1,01001020200,0.864406779661,143644.067796610179,187881.355932203383,72.372881355932,59.55,middle,42,White
2,01001020300,0.909836065574,135245.901639344258,169016.393442622939,69.483606557377,25.47,middle,30,White
3,01001020400,0.918604651163,158139.534883720917,199709.302325581404,79.372093023256,17.21,middle,30,White
4,01001020500,0.882121807466,196434.184675834957,239518.664047151280,88.713163064833,31.54,upper,30,White


In [16]:
# Join HMDA and Social Capital data on census tract
#### This will generate a lot of duplicate SC data

print(hmda.shape)
merged = hmda.merge(sc, on="census_tract", how="inner")

print(merged.shape)
maxrow()
merged.tail()

(72681, 9)
(116795, 18)


,census_tract,approval,loan_amount,property_value,income,tract_minority_population_percent,income_level,dti_level,derived_race,zip,num_below_p50,ec_zip,nbhd_ec_zip,exposure_grp_mem_zip,nbhd_exposure_zip,clustering_zip,volunteering_rate_zip,civic_organizations_zip
116755,56033000100,0.888297872340,2.161170212766e+05,3.332446808511e+05,97.351063829787,7.94,middle,50,White,82801,2114.728741884232,0.98988998,0.97591001,1.12671010,1.09155000,0.103260000,0.056839999,0.041290000000
116756,56033000200,0.887096774194,2.006451612903e+05,3.120967741935e+05,81.241935483871,9.00,middle,30,White,82801,2114.728741884232,0.98988998,0.97591001,1.12671010,1.09155000,0.103260000,0.056839999,0.041290000000
116757,56033000300,0.904761904762,2.524285714286e+05,3.927142857143e+05,107.780952380952,4.22,middle,50,White,82801,2114.728741884232,0.98988998,0.97591001,1.12671010,1.09155000,0.103260000,0.056839999,0.041290000000
116758,56033000400,0.823529411765,1.988502673797e+05,2.797593582888e+05,73.636363636364,14.45,middle,50,White,82801,2114.728741884232,0.98988998,0.97591001,1.12671010,1.09155000,0.103260000,0.056839999,0.041290000000
116759,56033000500,0.863799283154,3.592652329749e+05,6.628853046595e+05,154.007168458781,3.47,middle,30,White,82801,2114.728741884232,0.98988998,0.97591001,1.12671010,1.09155000,0.103260000,0.056839999,0.041290000000
116760,56033000600,0.905555555556,2.903333333333e+05,5.088888888889e+05,113.438888888889,10.93,middle,30,White,82801,2114.728741884232,0.98988998,0.97591001,1.12671010,1.09155000,0.103260000,0.056839999,0.041290000000
116761,56035000101,0.858490566038,2.393396226415e+05,3.685849056604e+05,108.018867924528,12.76,middle,50,White,82941,314.871205329895,1.09934000,1.35209000,1.28013000,1.38507000,0.094439998,0.119380000,0.039799999000
116762,56035000101,0.858490566038,2.393396226415e+05,3.685849056604e+05,108.018867924528,12.76,middle,50,White,83113,201.022537231445,1.01189990,1.15580000,1.13956000,1.11060000,0.111300000,0.107260000,0.022550000000
116763,56035000102,0.906336088154,2.641460055096e+05,3.958539944904e+05,114.490358126722,11.24,upper,50,White,82941,314.871205329895,1.09934000,1.35209000,1.28013000,1.38507000,0.094439998,0.119380000,0.039799999000
116764,56037970500,0.842857142857,1.708571428571e+05,2.192857142857e+05,82.371428571429,14.59,middle,42,White,82901,1722.641402721405,0.94694000,1.16171000,1.24769000,1.28729000,0.105750000,0.069870003,0.008949999700


In [17]:
merged.columns

Index(['census_tract', 'approval', 'loan_amount', 'property_value', 'income',
       'tract_minority_population_percent', 'income_level', 'dti_level',
       'derived_race', 'zip', 'num_below_p50', 'ec_zip', 'nbhd_ec_zip',
       'exposure_grp_mem_zip', 'nbhd_exposure_zip', 'clustering_zip',
       'volunteering_rate_zip', 'civic_organizations_zip'],
      dtype='object')

In [21]:
# Mode creates arrays. Use this fucntion to select the first mode if there are multiple
mode = lambda x: pd.Series.mode(x)[0]

merged2 = merged.groupby('zip', as_index=False).agg({
    "census_tract": mode,
    "approval": pd.Series.mean,
    "loan_amount": pd.Series.mean,
    "property_value": pd.Series.mean,
    "income": pd.Series.mean,
    'tract_minority_population_percent': pd.Series.mean,
    'income_level': mode,
    'dti_level': mode,
    "derived_race": mode, # check to see if this is in data
    "num_below_p50": pd.Series.mean,
#     "pop2018": pd.Series.mean,
    "ec_zip": pd.Series.mean,
    "nbhd_ec_zip": pd.Series.mean,
    "exposure_grp_mem_zip": pd.Series.mean,
    "nbhd_exposure_zip": pd.Series.mean,
    "clustering_zip": pd.Series.mean,
    "volunteering_rate_zip": pd.Series.mean,
    "civic_organizations_zip": pd.Series.mean
})
print(merged2.shape)
merged2.tail(20)

(14256, 18)


,zip,census_tract,approval,loan_amount,property_value,income,tract_minority_population_percent,income_level,dti_level,derived_race,num_below_p50,ec_zip,nbhd_ec_zip,exposure_grp_mem_zip,nbhd_exposure_zip,clustering_zip,volunteering_rate_zip,civic_organizations_zip
14236,99678,02070000100,0.857142857143,183571.428571428580,296428.571428571420,88.428571428571,94.360000000000,moderate,30,American Indian or Alaska Native,299.355529785156,0.89546001,0.86679000,0.97929001,0.97280997,0.198300000,0.188010010,0.0057899999
14237,99685,02016000200,0.677419354839,306612.903225806425,422741.935483870970,172.451612903226,71.340000000000,upper,30,White,334.078216552734,0.84441000,1.58431990,1.20505000,1.42857000,0.120640000,0.064510003,0.0099200001
14238,99686,02261000300,0.904255319149,236914.893617021269,303829.787234042538,110.361702127660,28.010000000000,upper,50,White,282.519705474377,1.04625000,0.74320000,1.00825000,0.86282003,0.121780000,0.123060000,0.0279200000
14239,99701,02090000100,0.843613132747,220031.268002905534,272755.255660224473,96.198555233525,33.061111111111,moderate,42,White,2654.180281341076,1.02661000,0.97815001,1.18138000,1.18961000,0.099399999,0.126629990,0.0328399990
14240,99705,02090001400,0.801023899296,242401.518187282549,294260.682865521580,94.051994716030,21.592000000000,middle,50,White,1686.666440248489,1.14628010,1.34328000,1.26501000,1.45051000,0.088320002,0.134079990,0.0113500000
14241,99709,02090000200,0.865772268646,247129.727657670126,307443.941848264891,101.261597954704,26.776250000000,upper,42,White,1932.823287963867,1.11244000,1.25776990,1.27690010,1.36932000,0.095579997,0.140440000,0.0144500000
14242,99712,02090001200,0.877537457799,267431.421065721137,334005.008520864358,110.414449827175,14.056666666667,upper,30,White,676.489173889160,1.16031000,1.37003000,1.26199000,1.42541000,0.089960001,0.149979990,0.0074300002
14243,99737,02240000400,0.820224719101,266910.112359550549,303202.247191011207,98.516853932584,13.910000000000,middle,30,White,643.618768692017,1.05997000,1.39443000,1.22126000,1.29827000,0.105940000,0.092210002,0.0323100020
14244,99738,02240000100,0.800000000000,168333.333333333343,191000.000000000000,91.266666666667,43.530000000000,moderate,30,White,34.114139556885,0.90772998,1.03549000,1.08611000,1.06710990,0.166300000,0.149990010,0.0143800000
14245,99752,02188000200,0.793103448276,265344.827586206899,360862.068965517217,129.620689655172,78.920000000000,middle,30,American Indian or Alaska Native,675.340095520020,0.88957000,0.93401998,0.99540001,1.01502000,0.208989990,0.150800000,0.0123800000


In [26]:
# Cross check that the CT in the final feature set were also in the original HMDA data
hmda_ct = pd.DataFrame(hmda["census_tract"])
val = hmda_ct.merge(merged2, on="census_tract", how="inner")
print(val.shape)
val.head()

,census_tract,zip,approval,loan_amount,property_value,income,tract_minority_population_percent,income_level,dti_level,derived_race,num_below_p50,ec_zip,nbhd_ec_zip,exposure_grp_mem_zip,nbhd_exposure_zip,clustering_zip,volunteering_rate_zip,civic_organizations_zip
0,01001020100,36067,0.822122348245,171607.159908452828,220225.288111522183,82.637134779485,27.158461538462,middle,30,White,3396.898228928447,0.69064999,0.62941003,0.80005997,0.74044001,0.104340000,0.042989999,0.01568
1,01001020400,36066,0.856671727561,184503.956627729000,230372.686645345151,84.977357215932,20.938000000000,middle,30,White,1261.963180541992,0.77508003,0.73109001,0.85949999,0.90544999,0.101710000,0.045180000,0.01684
2,01001020802,36022,0.828298675571,189047.226406593225,237536.176186322555,85.626492739571,23.336000000000,middle,30,White,1141.169984102249,0.82287002,1.00111000,0.89157999,1.00566010,0.099090002,0.047090001,0.00397
3,01001021100,36703,0.589568411387,93054.803445712547,122503.128689492311,140.210220822948,79.320000000000,moderate,100,Black or African American,3603.374973297119,0.42645001,0.24936999,0.45030999,0.29679999,0.163940000,0.039850000,0.01849
4,01003010100,36502,0.648167264497,126960.477163280331,169034.430576056213,68.591232938047,33.054444444444,middle,30,White,3165.831730842590,0.42914000,0.28062001,0.41001999,0.31794000,0.145480010,0.025850000,0.01148


In [28]:
merged4.shape

(14256, 18)

### Export data

In [ ]:
# merged.to_parquet(path="data/output/black_features_20230414_allCT", index=None)
# Too many duplicates. Don't use in model

In [31]:
merged2.to_parquet("data/output/all_features_aggZip_"+str(today), index=None)
merged2.to_csv("data/output/all_features_aggZip_"+str(today)+".csv", index=None, header=True)